In [2]:
import numpy as np
import prettytable
import logging 
import time
import ipywidgets as widgets
from IPython.display import clear_output, display
        
class Node:
    def __init__(self, m, c, bA_pos, bA_move, bA_m, bA_c, bB_pos, bB_move, bB_m, bB_c, step, cost, parent, setting, boatSetting):
        self.state = {  'm' : m, 'c' : c, 'left_m':  setting['totalM'] - m, 'left_c' : setting['totalC'] - c,
                        'bA' : bA_pos, 'bB' : bB_pos # bA_pos: boat A's position，right : 1 (right -> left), left:-1 (left -> right)
                    }
        self.bAMove = bA_move #bAMove : 1(move) 
        self.bBMove = bB_move
                      
        self.boatA = {'bA_m' : bA_m, 'bA_c' : bA_c} # Missionaries on boat A, cannibals on boat A
        self.boatB = {'bB_m' : bB_m, 'bB_c' : bB_c} # Missionaries on boat B, cannibals on boat B
        
        h = m  + c  - bA_pos * boatSetting['A']['capacity'] * bA_move - bB_pos * boatSetting['B']['capacity'] * bB_move
        if h < 0 or m + c == 0:
            h = 0
        self.h = h
        f = h + step if setting['costOrStep'] else h + cost # f(n) = g(n) + h(n)
        self.data_nof = np.array([cost, step])
        self.data = np.array([cost, step, f]) # cost: total cost, step: number of steps
        self.parent = parent

def is_safe(right_m, right_c, left_m, left_c, setting):
    m = right_m
    c = right_c
    total_m = right_m + left_m
    total_c = right_c + left_c
    
    if m < 0 or c < 0 or left_m < 0 or left_c < 0 or total_m != setting['totalM'] or total_c != setting['totalC']: 
        # the number of people cannot be less than 0
        return False

    if (m < c and m > 0) or ( (left_m < left_c) and left_m > 0): 
        # cannibals <= missionaries  unless missionaries == 0 (right/left bank)
        return False
   
    if (m == 0 and left_m >= left_c) or (left_m == 0 and m >= c) or (m >= c and left_m >= left_c) or (m == 0 and left_m == 0): 
        return True

def boat_actions (id, boatSetting):
    set_boat_operation = [] 
    for i in range (boatSetting[id]['capacity'] + 1):
        if (i == 0): # 船上只有野人
            for j in range(0, boatSetting[id]['capacity'] + 1):  
                set_boat_operation.append([i, j, boatSetting[id]['cost'], boatSetting[id]['step']])

        else: #船上有傳教士
            for j in range (0, i + 1):
                if i + j <= boatSetting[id]['capacity']: 
                    set_boat_operation.append([i, j, boatSetting[id]['cost'], boatSetting[id]['step'] ]) 
    print(f"boat{id} 有{len(set_boat_operation)}種動作，各為: {set_boat_operation}\n")
    return set_boat_operation

def sortNode (dataList, cost_or_step, useAStar):
    
    cost_or_step_or_aStar = 2 if useAStar else cost_or_step
    for i in range(1, len(dataList)):
        for j in range(i, len(dataList)):
                change = 0
                if (dataList[i].data[cost_or_step_or_aStar] < dataList[j].data[cost_or_step_or_aStar]):
                    change = 1
                elif (dataList[i].data[cost_or_step_or_aStar] == dataList[j].data[cost_or_step_or_aStar]) and cost_or_step_or_aStar != 0: # 0 cost, 1 step, 2 aStar
                    if (dataList[i].data[0] < dataList[j].data[0]):
                        change = 1
                    elif (dataList[i].data[0] == dataList[j].data[0]) and dataList[i].data[2] < dataList[j].data[2]:
                        change = 1
                if change:
                    temp = dataList[i]
                    dataList[i] = dataList[j]
                    dataList[j] = temp

def get_all_children(curr_node, boatA_operations, boatB_operations, setting, boatSetting, useAStar, limit):
    successor = []
    right_m = curr_node.state['m']
    right_c = curr_node.state['c']
    left_m = curr_node.state['left_m']
    left_c = curr_node.state['left_c']

    for boatA_operation in boatA_operations:

        if (boatA_operation[0] + boatA_operation[1] != 0):
            bA_state = np.array([curr_node.state['bA'], boatSetting['A']['cost'], boatSetting['A']['step'], 1]) # 1: moving
        else:
            bA_state = [-curr_node.state['bA'], 0, 0, 0] # bA_pos, bACost, bATime, bAmove

        if curr_node.state['bA'] > 0: 
            # boat A on the right bank
            boatA_m_to_left = boatA_operation[0]
            boatA_c_to_left = boatA_operation[1]
        else: 
            # boat A on the left bank
            boatA_m_to_left = -boatA_operation[0]
            boatA_c_to_left = -boatA_operation[1]
        
        for boatB_operation in boatB_operations:
                    
            if (boatB_operation[0] + boatB_operation[1] != 0):
                bB_state = np.array([curr_node.state['bB'], boatSetting['B']['cost'], boatSetting['B']['step'], 1]) # 1: moving    
            else :
                bB_state = [-curr_node.state['bB'], 0, 0, 0] # bB_pos, bBCost, bBTime, bBmove

            if curr_node.state['bB'] > 0: 
            # boat B on the right bank
                boatB_m_to_left = boatB_operation[0]
                boatB_c_to_left = boatB_operation[1]
            else: 
                # boat B on the left bank
                boatB_m_to_left = -boatB_operation[0]
                boatB_c_to_left = -boatB_operation[1]

            right_m_bB = right_m - boatA_m_to_left - boatB_m_to_left # 右岸剩下的傳教士 (右岸剩下的傳教士人數 -/+ boatB載的傳教士人數)
            right_c_bB = right_c - boatA_c_to_left - boatB_c_to_left # 右岸剩下的食人族 (右岸剩下的食人族人數 -/+ boatB載的食人族人數)
            left_m_bB = left_m + boatA_m_to_left + boatB_m_to_left # 左岸剩下的傳教士 (左岸原本的傳教士人數 +/- boatB載的傳教士人數)
            left_c_bB = left_c + boatA_c_to_left + boatB_c_to_left # 左岸剩下的食人族 (左岸原本的食人族人數 +/- boatB載的食人族人數)
            
            if is_safe(right_m_bB, right_c_bB, left_m_bB, left_c_bB, setting) and not(bA_state[3] == 0 and bB_state[3] == 0):
                
                step = curr_node.data[1] + 1 #bA_state[2] + bB_state[2] - min(bA_state[2], bB_state[2]) #原本已經走的step加上boat A, boat B的step 減掉他們重疊的step
                cost = curr_node.data[0] + bA_state[1] + bB_state[1]
                if (step <= limit['step']['max']  and cost <= limit['cost']['max']):
                    successor.append(Node(right_m_bB, right_c_bB, -bA_state[0], bA_state[3], boatA_operation[0], boatA_operation[1], -bB_state[0], bB_state[3], boatB_operation[0], boatB_operation[1], step, cost, curr_node, setting, boatSetting))
    
    sortNode (successor, setting['costOrStep'], useAStar)

    return successor

def printTable(dataList):
    table = prettytable.PrettyTable()
    table.field_names = ["step", "left m", "left c", "boatA DIRE", "boatA [m c]", "boatB DIRE", "boatB [m c]" , "right m", "right c", "cost", "AStar_h(n)"]

    while dataList:
        node = dataList.pop()
        if (node.state['bA'] == -1 and node.bAMove == 1):
            boatA_direction = "L <- R"
        elif(node.state['bA'] == -1 and node.bAMove == 0):
            boatA_direction = "L (X)"
        elif (node.state['bA'] == 1 and node.bAMove == 1):
            boatA_direction = "L -> R"
        else :
            boatA_direction = "R (X)"

        if (node.state['bB'] == -1 and node.bBMove == 1):
            boatB_direction = "L <- R"
        elif(node.state['bB'] == -1 and node.bBMove == 0):
            boatB_direction = "L (X)"
        elif (node.state['bB'] == 1 and node.bBMove == 1):
            boatB_direction = "L -> R"
        else :
            boatB_direction = "R (X)"

        table.add_row([ node.data[1], node.state['left_m'], node.state['left_c'], 
                        boatA_direction, list(node.boatA.values()), boatB_direction, list(node.boatB.values()), 
                        node.state['m'], node.state['c'], node.data[0], node.h])
    with open('Q2_result.txt', 'a') as w:
        w.write(str(table))    
    #print(table)

def logging_file(open_list, close_list):
    logging.basicConfig(filename= "basic.log", level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

    logging.info('********open list********, len = {}'.format(len(open_list)))
    for open_data in open_list:
        logging.debug('{}\n> cost = {}, step = {}, f(n) = {}\n> {}\n> {}'.format(   open_data.state,
                                                                                    open_data.data[0], open_data.data[1], open_data.data[2], 
                                                                                    open_data.boatA, open_data.boatB))
    logging.info('********close list********, len = {}'.format(len(close_list)))
    for close_data in close_list:
        logging.debug('{}\n> cost = {}, step = {}, f(n) = {}\n> {}\n> {}'.format(   close_data.state,
                                                                                    close_data.data[0], close_data.data[1], close_data.data[2], 
                                                                                    close_data.boatA, close_data.boatB))
    logging.info('--------------------------------------------------------------------------------------------------------')

def calculate_path(curr_node):
    result = []
    result.append(curr_node)
    parent_node = curr_node.parent
    while (parent_node):
        result.append(parent_node)
        parent_node = parent_node.parent
    return result

def uniform_cost_search(start_node, setting, boatSetting, limit):
    open_list = []
    close_list = []
    open_list.append(start_node)
    useAStar = 0
    count_open = 1
    count_close = 0
    start_time = time.time()
    
    while True:
        if not open_list or (time.time() - start_time > 120):
            
            print ("no solution!!!\n")

            with open('Q2_result.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            break
        
        curr_node = open_list.pop()

        if (curr_node.state['m'] == 0 and curr_node.state['c'] == 0): # find goal
            
            result = calculate_path(curr_node)
            printTable(result)
            print(f"count_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
        
            with open('Q2_result.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            break

        close_list.append(curr_node)
        count_close = count_close + 1
        children = get_all_children(curr_node, boatA_operations, boatB_operations, setting, boatSetting, useAStar, limit) # cost: 0 , time: 1
        
        for child in children:
            sameNode_in_open_list = 0
            sameNode_in_close_list = 0
            
            for i in range(len(close_list)):
                if np.array_equal(child.state, close_list[i].state) and np.array_equal(child.data_nof, close_list[i].data_nof):
                    sameNode_in_close_list = 1
                    break
            
            for j in range(len(open_list)):
                if np.array_equal(child.state, open_list[j].state):
                    sameNode_in_open_list = 1
                    if child.data[ setting['costOrStep'] ] < open_list[j].data[ setting['costOrStep'] ]:
                        open_list.pop(j)
                    break
            
            if (sameNode_in_close_list != 1 and sameNode_in_open_list != 1):
                open_list.append(child)
                count_open =  1 + count_open
        sortNode(open_list, setting['costOrStep'], useAStar)
        
        #logging_file(open_list, close_list)

def AStar_algorithm(start_node, setting, boatSetting, limit):
    open_list = []
    close_list = []
    open_list.append(start_node)
    useAStar = 1
    count_open = 1
    count_close = 0

    while True:
        if not open_list:
            print ("no solution!!! --- open_list is empty\n")
            print(f"count_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            with open('Q2_result.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            
            break
        
        curr_node = open_list.pop()
        
        if (curr_node.state['m'] == 0 and curr_node.state['c'] == 0):
            
            result = calculate_path(curr_node)
            printTable(result)
            print(f"count_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            with open('Q2_result.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            
            break

        close_list.append(curr_node)
        count_close = count_close + 1
        children = get_all_children(curr_node, boatA_operations, boatB_operations, setting, boatSetting, useAStar, limit) 

        for child in children:
            sameNode_in_open_list = 0
            sameNode_in_close_list = 0

            for i in range(len(open_list)):
                if np.array_equal(child.state, open_list[i].state):
                    if child.data[2] < open_list[i].data[2]: # beter => remove older, append newer, AStar_f
                        open_list.pop(i)
                    else : # newer no better
                        sameNode_in_open_list = 1
                    break

            if (sameNode_in_open_list != 1):
                for j in range(len(close_list)):
                    if np.array_equal(child.state, close_list[j].state):
                        if child.data[2] < close_list[j].data[2]: # beter => remove older, append newer, AStar_f
                            close_list.pop(j)
                        else : # newer no better
                            sameNode_in_close_list = 1
                        break

                if (sameNode_in_close_list != 1):
                    open_list.append(child)
                    count_open =  1 + count_open

        sortNode(open_list, setting['costOrStep'], useAStar)
        #logging_file(open_list, close_list)

boatSetting = dict()
boatSetting['A'] = { 'capacity': 2, 'cost': 3,  'step' : 1, 'pos' : 1} # pos : right bank: 1, left bank: -1
boatSetting['B'] = { 'capacity': 3, 'cost': 25, 'step' : 1, 'pos' : 1}
boatA_operations = boat_actions('A', boatSetting)
boatB_operations = boat_actions('B', boatSetting)

limit = dict()
limit['cost'] = {'max': 1000}
limit['step'] = {'max': 100}

totalM = 10
totalC = 10
setting = {'totalM': totalM, 'totalC' : totalC, 'costOrStep' : 0}
start_node = Node(setting['totalM'], setting['totalC'], 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, None, setting, boatSetting) # 初始狀態節點


with open('Q2_result.txt', 'a+', encoding='utf-8') as w:
    w.truncate(0)

for m in range(3): #3
    for n in range (3, 11): #3, 11
        
        for i in range (2):
            setting = {'totalM': m+n, 'totalC' : n, 'costOrStep' : i}
            start_node = Node(m + n, n, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, None, setting, boatSetting) # 初始狀態節點
            goal_node = [0, 0]

            with open('Q2_result.txt', 'a') as w:
                w.write(f"******************\ntotalM: {m+n}, totalC: {n}, costOrStep: {i}\n")

            print(f"Astar  totalM: {m+n}, totalC: {n}, costOrStep: {i}")
            start_time = time.time()
            AStar_algorithm (start_node, setting, boatSetting, limit)
            end_time = time.time()
            with open('Q2_result.txt', 'a') as w:
                w.write(f", AStar exe time : { round(end_time - start_time, 3)}\n\n")
            
            print(f"UCS!!  totalM: {m+n}, totalC: {n}, costOrStep: {i}")
            start_time = time.time()
            uniform_cost_search(start_node, setting, boatSetting, limit)
            end_time = time.time()
            with open('Q2_result.txt', 'a') as w:
                w.write(f", UCS exe time: { round(end_time - start_time, 3)}\n\n")

boatA 有6種動作，各為: [[0, 0, 3, 1], [0, 1, 3, 1], [0, 2, 3, 1], [1, 0, 3, 1], [1, 1, 3, 1], [2, 0, 3, 1]]

boatB 有9種動作，各為: [[0, 0, 25, 1], [0, 1, 25, 1], [0, 2, 25, 1], [0, 3, 25, 1], [1, 0, 25, 1], [1, 1, 25, 1], [2, 0, 25, 1], [2, 1, 25, 1], [3, 0, 25, 1]]

Astar  totalM: 3, totalC: 3, costOrStep: 0
count_open: 49, count_close: 24, open+close: 73
UCS!!  totalM: 3, totalC: 3, costOrStep: 0
count_open: 106, count_close: 64, open+close: 170
Astar  totalM: 3, totalC: 3, costOrStep: 1
count_open: 29, count_close: 3, open+close: 32
UCS!!  totalM: 3, totalC: 3, costOrStep: 1
count_open: 63, count_close: 35, open+close: 98
Astar  totalM: 4, totalC: 4, costOrStep: 0
count_open: 66, count_close: 32, open+close: 98
UCS!!  totalM: 4, totalC: 4, costOrStep: 0
count_open: 165, count_close: 92, open+close: 257
Astar  totalM: 4, totalC: 4, costOrStep: 1
count_open: 47, count_close: 5, open+close: 52
UCS!!  totalM: 4, totalC: 4, costOrStep: 1
count_open: 106, count_close: 71, open+close: 177
Astar  totalM

In [ ]:
with open('Q2_result.txt', 'a+', encoding='utf-8') as w:
    w.truncate(0)

for m in range(3): #3
    for n in range (3, 11): #3, 11
        
        for i in range (2):
            setting = {'totalM': m+n, 'totalC' : n, 'costOrStep' : i}
            start_node = Node(m + n, n, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, None, setting, boatSetting) # 初始狀態節點
            goal_node = [0, 0]

            with open('Q2_result.txt', 'a') as w:
                w.write(f"******************\ntotalM: {m+n}, totalC: {n}, costOrStep: {i}\n")

            print(f"Astar  totalM: {m+n}, totalC: {n}, costOrStep: {i}")
            start_time = time.time()
            AStar_algorithm (goal_node, start_node, setting, boatSetting, limit)
            end_time = time.time()
            with open('Q2_result.txt', 'a') as w:
                w.write(f", AStar exe time : { round(end_time - start_time, 3)}\n\n")
            
            print(f"UCS!!  totalM: {m+n}, totalC: {n}, costOrStep: {i}")
            start_time = time.time()
            uniform_cost_search(goal_node, start_node, setting, boatSetting, limit)
            end_time = time.time()
            with open('Q2_result.txt', 'a') as w:
                w.write(f", UCS exe time: { round(end_time - start_time, 3)}\n\n")